In [1]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

C:\Users\Star\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [3]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

In [4]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)
print(train)

angry completed
fear completed
happy completed
neutral completed
sad completed
surprise completed
                              image     label
0          images/train\angry\0.jpg     angry
1          images/train\angry\1.jpg     angry
2         images/train\angry\10.jpg     angry
3       images/train\angry\1015.jpg     angry
4       images/train\angry\1016.jpg     angry
...                             ...       ...
2231  images/train\surprise\894.jpg  surprise
2232  images/train\surprise\909.jpg  surprise
2233  images/train\surprise\960.jpg  surprise
2234  images/train\surprise\964.jpg  surprise
2235  images/train\surprise\972.jpg  surprise

[2236 rows x 2 columns]


In [5]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)
print(test)

angry completed
fear completed
happy completed
neutral completed
sad completed
surprise completed
                             image     label
0       images/test\angry\1024.jpg     angry
1       images/test\angry\1071.jpg     angry
2       images/test\angry\1106.jpg     angry
3       images/test\angry\1124.jpg     angry
4       images/test\angry\1127.jpg     angry
..                             ...       ...
983  images/test\surprise\9806.jpg  surprise
984  images/test\surprise\9830.jpg  surprise
985  images/test\surprise\9853.jpg  surprise
986  images/test\surprise\9878.jpg  surprise
987   images/test\surprise\993.jpg  surprise

[988 rows x 2 columns]


In [6]:
from tqdm.notebook import tqdm

# Data Preprocessing

In [7]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale=True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

# Feature Extraction

In [8]:
train_features = extract_features(train['image'])
test_features = extract_features(test['image'])

  0%|          | 0/2236 [00:00<?, ?it/s]

C:\Users\Star\Anaconda3\lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


  0%|          | 0/988 [00:00<?, ?it/s]

# Normalization

In [9]:
x_train = train_features / 255.0
x_test = test_features / 255.0

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

# Building the Neural Network Model

In [11]:
# sequential model
model = Sequential()

# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# flattening
model.add(Flatten())

# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(7, activation='softmax'))

# model compilation
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )



# Training the Model

In [12]:
model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
18/18 [==============================] - 211s 11s/step - loss: 1.8464 - accuracy: 0.2084 - val_loss: 1.8841 - val_accuracy: 0.1296
Epoch 2/100
18/18 [==============================] - 207s 12s/step - loss: 1.7953 - accuracy: 0.2267 - val_loss: 1.8596 - val_accuracy: 0.1822
Epoch 3/100
18/18 [==============================] - 164s 9s/step - loss: 1.7850 - accuracy: 0.2379 - val_loss: 1.8521 - val_accuracy: 0.1822
Epoch 4/100
18/18 [==============================] - 148s 8s/step - loss: 1.7799 - accuracy: 0.2258 - val_loss: 1.8441 - val_accuracy: 0.1822
Epoch 5/100
18/18 [==============================] - 143s 8s/step - loss: 1.7838 - accuracy: 0.2227 - val_loss: 1.8483 - val_accuracy: 0.1822
Epoch 6/100
18/18 [==============================] - 137s 8s/step - loss: 1.7726 - accuracy: 0.2106 - val_loss: 1.8436 - val_accuracy: 0.1822
Epoch 7/100
18/18 [==============================] - 147s 8s/step - loss: 1.7771 - accuracy: 0.2272 - val_loss: 1.8462 - val_accuracy: 0.1822
Epoc

# Saving the Model

In [13]:
model_json = model.to_json()
with open("emotiondetector.json", 'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

C:\Users\Star\Anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
